In [6]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.Collecting langchain==0.0.284 (from -r requirements.txt (line 1))
  Using cached langchain-0.0.284-py3-none-any.whl.metadata (14 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached streamlit-1.22.0-py2.py3-none-any.whl.metadata (7.3 kB)
  Using cached tiktoken-0.4.0.tar.gz (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'



ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.8.0, 1.8.0.post1, 1.9.0, 1.9.0.post1, 1.10.0, 1.11.0, 1.11.0.post1, 1.12.0, 1.13.0, 1.13.1)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-cpu==1.7.4


In [1]:
%pip install langchain-huggingface huggingface_hub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:

from dotenv import load_dotenv
import os
load_dotenv("E:\\AI-Buildathon\\.env")
api_key = os.getenv("GOOGLE_GENAI_API_KEY")




In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, api_key=api_key)





In [39]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=PyPDFLoader("E:\\AI-Buildathon\\WHO.pdf")
document= loader.load()




In [40]:
def text_splitter(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    text_chunk = text_splitter.split_documents(document)
    return text_chunk

text_chunk= text_splitter(document)
print(f"Number of Chunks: {len(text_chunk)}")

Number of Chunks: 857


In [30]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\pytho3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\acer\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
load_dotenv("E:\\AI-Buildathon\\.env")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [27]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("test")

In [28]:
%pip install -qU "langchain-pinecone"


Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
from langchain_pinecone import PineconeVectorStore

docstore = PineconeVectorStore.from_documents(
    documents=text_chunk,
    embedding=embedding_model,
    index_name="test",
)


In [41]:
docsearch= PineconeVectorStore.from_existing_index(
    embedding=embedding_model,
    index_name="test"
)

In [42]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [43]:
retrieved_docs = retriever.invoke("What is Antenatal Care?")
retrieved_docs

[Document(id='d34bfc67-b2ba-4fa8-8f00-098185e168ac', metadata={'author': 'ALLSOPP, Caroline E.', 'creationdate': '2025-02-25T13:49:54-05:00', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'moddate': '2025-05-20T09:21:28+00:00', 'page': 22, 'page_label': '23', 'producer': 'Adobe PDF Library 15.0', 'source': 'E:\\AI-Buildathon\\WHO.pdf', 'total_pages': 208, 'trapped': '/False'}, page_content='Context-specific recommendation - Research  \nGroup antenatal care provided by qualified health-care professionals may be offered as an alternative \nto individual antenatal care for pregnant women in the context of rigorous research, depending on \na woman’s preferences and provided that the infrastructure and resources for delivery of group \nantenatal care are available.\xa0\n ● With the group ANC model, the first visit for all pregnant women is an individual visit. Then at subsequent \nvisits, the usual individual pregnancy health assessment, held in a private examination area, is integrated \ni

In [73]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import (create_stuff_documents_chain)

system_prompt=(
    "You are MaBondhu AI, a clinical assistant for maternal health. "
        "Provide safe, evidence-based, concise guidance. Use the following pieces of retrieved context to answer the question at the end. "
        "Language: {language}.\n\n"
        "Rules:\n"
        "1) If this is potentially life-threatening, say 'EMERGENCY' and recommend immediate referral.\n"
        "2) If unsure, recommend human-in-the-loop evaluation and list needed vitals/tests.\n"
        "3) Include 1–2 specific, actionable next steps (e.g., 'go to nearest clinic', 'take iron supplement 60mg').\n"
        "4) Provide concise Bangla/English text depending on language.\n\n"
        "5) If you don't know the answer, say 'Sorry, I don't know about it'.\n\n"
        "Answer now in clear {language} with citations (e.g., [WHO Section 3])."
        "\n\n"
        "Context:\n{context}\n\n"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [74]:
question_answering_chain = create_stuff_documents_chain(llm,prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [ ]:
response = rag_chain.invoke({
    "input": "What are common diseases for a mother?",
    "language": "English"
})

print(response['answer'])

Sorry, I don't know about it.
